## THOR Tutorials - Generating Ephemeris

In [1]:
import numpy as np
import pandas as pd
from astropy.time import Time

from astroquery.jplhorizons import Horizons

from thor.orbits.ephemeris import generateEphemeris
from thor.observatories import getObserverState

/Users/moeyensj/projects/thor/thor/thor/orbits/kepler.py:55: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  e_vec = ((v_mag**2 - mu / r_mag) * r - (np.dot(r, v)) * v) / mu
/Users/moeyensj/projects/thor/thor/thor/orbits/kepler.py:77: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  trueAnom_deg = np.degrees(np.arccos(np.dot(e_vec, r) / (e * r_mag)))
/Users/moeyensj/projects/thor/thor/thor/orbits/kepler.py:78: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  if np.dot(r, v) < 0:
/Users/moeyensj/projects/thor/thor/thor/orbits/propagate/universal.py:46: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  rv_mag = np.dot(r, v) / r_mag


Let's first get some orbits from JPL's Horizons service. 

Create a list of your favorite minor planets:

In [2]:
targets = [
    "Ceres",
    "Eros",
    "Duende",
    "Amor",
    "Aten"
]

We will query Horizons for state vectors only for a single epoch but once we have those state vectors, we can then generate ephemeris for the remaining 'observation' times.

In [3]:
start_mjd = 57580.0
end_mjd = start_mjd + 30.0
step_mjd = 1.0

observation_times = Time(
    np.arange(start_mjd, end_mjd + step_mjd, step_mjd),
    scale="utc",
    format="mjd"
)

In [4]:
observation_times[0].isot

'2016-07-11T00:00:00.000'

In [5]:
orbits = []
t0 = []
for name in targets:
    obj = Horizons(id=name, epochs=observation_times[0], location="@sun")
    df = obj.vectors().to_pandas()
    orbit = df[["x", "y", "z", "vx", "vy", "vz"]].values
    orbits.append(orbit[0, :])
    
    t0.append(observation_times[0])
    
orbits = np.array(orbits)
t0 = Time(t0)

Now we have an array of cartesian state vectors and an array of epochs at which those state vectors are defined. 

In [6]:
orbits

array([[ 2.86107593e+00,  4.05024279e-01, -5.14638072e-01,
        -1.70128971e-03,  9.56388437e-03,  6.14460690e-04],
       [ 1.22114897e+00, -1.28550678e+00,  5.42153575e-02,
         7.67767895e-03,  8.20474710e-03,  2.09775644e-03],
       [-4.87939838e-01,  8.29808004e-01, -8.82658763e-02,
        -1.50320455e-02, -7.25226377e-03,  2.93262913e-03],
       [ 1.92186805e+00,  1.68045745e+00, -4.10309701e-01,
        -7.34528933e-03,  4.50055820e-03, -7.03468368e-04],
       [ 9.37710396e-01,  5.85051521e-02, -3.11307133e-01,
         1.44880779e-03,  1.68644847e-02, -2.31265764e-03]])

In [7]:
t0

<Time object: scale='utc' format='mjd' value=[57580. 57580. 57580. 57580. 57580.]>

The last remaining component we need is to designated a few observers

In [8]:
observers = {
    "I11" : observation_times[1:],
    "I41" : observation_times[1:] + 1/24
}

In the above hypothetical scenario, I11 is observing at the remaining observation times we defined earlier, while I41 is doing the same but observing an hour later. 

We now have everything we need to generate ephemeris for each object. `THOR` currently has two supported ephemeris backends:
- "THOR" : A universal 2-body propagator with optional light time correction for ephemeris generation. 
- "PYOORB" : OpenOrb's python wrapper with both 2- and N-body propagation and ephemeris generation. 

By default `THOR` uses the "THOR" backend, but backends are easily configurable. 

In [9]:
ephemeris = generateEphemeris(
    orbits,
    t0,
    observers
)

In [10]:
ephemeris

,orbit_id,observatory_code,mjd_utc,RA_deg,Dec_deg,vRAcosDec,vDec,r_au,delta_au,light_time,...,obj_z,obj_vx,obj_vy,obj_vz,obs_x,obs_y,obs_z,obs_vx,obs_vy,obs_vz
0,0,I11,57581.000000,30.178725,1.448805,0.233286,0.047574,2.934633,2.909170,0.016802,...,-0.514031,-0.001734,0.009559,0.000620,0.345325,-0.956164,0.000022,0.016026,0.005644,0.000061
1,0,I11,57582.000000,30.406713,1.495574,0.230460,0.045992,2.934183,2.895588,0.016724,...,-0.513408,-0.001768,0.009554,0.000626,0.361174,-0.950244,0.000021,0.015928,0.005914,0.000060
2,0,I11,57583.000000,30.631843,1.540755,0.227588,0.044401,2.933732,2.881982,0.016645,...,-0.512778,-0.001801,0.009549,0.000632,0.376920,-0.944057,0.000021,0.015826,0.006182,0.000059
3,0,I11,57584.000000,30.854068,1.584339,0.224668,0.042802,2.933278,2.868356,0.016566,...,-0.512143,-0.001835,0.009544,0.000638,0.392559,-0.937605,0.000021,0.015719,0.006448,0.000058
4,0,I11,57585.000000,31.073341,1.626317,0.221700,0.041195,2.932822,2.854710,0.016487,...,-0.511501,-0.001868,0.009539,0.000644,0.408087,-0.930890,0.000020,0.015608,0.006712,0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4,I41,57606.041667,81.605168,7.659833,0.802486,0.054598,1.062705,1.271238,0.007342,...,-0.340272,-0.005148,0.014977,0.000038,0.701768,-0.732320,0.000053,0.012245,0.011702,0.000061
296,4,I41,57607.041667,82.401143,7.714334,0.800741,0.052728,1.065132,1.271662,0.007345,...,-0.340192,-0.005365,0.014858,0.000122,0.713810,-0.720370,0.000052,0.012045,0.011909,0.000061
297,4,I41,57608.041667,83.195376,7.766968,0.799042,0.050858,1.067527,1.271993,0.007346,...,-0.340028,-0.005579,0.014736,0.000205,0.725648,-0.708217,0.000051,0.011842,0.012113,0.000060
298,4,I41,57609.041667,83.987914,7.817736,0.797391,0.048990,1.069889,1.272232,0.007348,...,-0.339782,-0.005790,0.014611,0.000287,0.737278,-0.695863,0.000050,0.011636,0.012314,0.000059


In [11]:
backend_kwargs = {
    "light_time" : True,
    "lt_tol" : 1e-10,
    "max_iter" : 1000,
    "tol" : 1e-15
}

ephemeris_thor = generateEphemeris(
    orbits,
    t0,
    observers,
    backend="THOR",
    backend_kwargs=backend_kwargs
)

In [12]:
ephemeris_thor

,orbit_id,observatory_code,mjd_utc,RA_deg,Dec_deg,vRAcosDec,vDec,r_au,delta_au,light_time,...,obj_z,obj_vx,obj_vy,obj_vz,obs_x,obs_y,obs_z,obs_vx,obs_vy,obs_vz
0,0,I11,57581.000000,30.178725,1.448805,0.233286,0.047574,2.934633,2.909170,0.016802,...,-0.514031,-0.001734,0.009559,0.000620,0.345325,-0.956164,0.000022,0.016026,0.005644,0.000061
1,0,I11,57582.000000,30.406713,1.495574,0.230460,0.045992,2.934183,2.895588,0.016724,...,-0.513408,-0.001768,0.009554,0.000626,0.361174,-0.950244,0.000021,0.015928,0.005914,0.000060
2,0,I11,57583.000000,30.631843,1.540755,0.227588,0.044401,2.933732,2.881982,0.016645,...,-0.512778,-0.001801,0.009549,0.000632,0.376920,-0.944057,0.000021,0.015826,0.006182,0.000059
3,0,I11,57584.000000,30.854068,1.584339,0.224668,0.042802,2.933278,2.868356,0.016566,...,-0.512143,-0.001835,0.009544,0.000638,0.392559,-0.937605,0.000021,0.015719,0.006448,0.000058
4,0,I11,57585.000000,31.073341,1.626317,0.221700,0.041195,2.932822,2.854710,0.016487,...,-0.511501,-0.001868,0.009539,0.000644,0.408087,-0.930890,0.000020,0.015608,0.006712,0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4,I41,57606.041667,81.605168,7.659833,0.802486,0.054598,1.062705,1.271238,0.007342,...,-0.340272,-0.005148,0.014977,0.000038,0.701768,-0.732320,0.000053,0.012245,0.011702,0.000061
296,4,I41,57607.041667,82.401143,7.714334,0.800741,0.052728,1.065132,1.271662,0.007345,...,-0.340192,-0.005365,0.014858,0.000122,0.713810,-0.720370,0.000052,0.012045,0.011909,0.000061
297,4,I41,57608.041667,83.195376,7.766968,0.799042,0.050858,1.067527,1.271993,0.007346,...,-0.340028,-0.005579,0.014736,0.000205,0.725648,-0.708217,0.000051,0.011842,0.012113,0.000060
298,4,I41,57609.041667,83.987914,7.817736,0.797391,0.048990,1.069889,1.272232,0.007348,...,-0.339782,-0.005790,0.014611,0.000287,0.737278,-0.695863,0.000050,0.011636,0.012314,0.000059


In [13]:
backend_kwargs = {
    "dynamical_model" : "2",
}

ephemeris_pyoorb = generateEphemeris(
    orbits,
    t0,
    observers,
    backend="PYOORB",
    backend_kwargs=backend_kwargs
)

In [14]:
ephemeris_pyoorb

,orbit_id,observatory_code,mjd_utc,RA_deg,Dec_deg,vRAcosDec,vDec,r_au,delta_au,obj_x,obj_y,obj_z,obj_vx,obj_vy,obj_vz,obs_x,obs_y,obs_z
0,0,I11,57581.000000,30.178666,1.448834,0.233827,0.047578,2.934633,2.909183,2.859387,0.414425,-0.514031,-0.001734,0.009559,0.000620,0.345312,-0.956168,0.000022
1,0,I11,57582.000000,30.406657,1.495604,0.231000,0.045996,2.934183,2.895601,2.857636,0.423983,-0.513408,-0.001768,0.009554,0.000626,0.361162,-0.950249,0.000021
2,0,I11,57583.000000,30.631790,1.540786,0.228125,0.044406,2.933732,2.881995,2.855851,0.433535,-0.512778,-0.001801,0.009549,0.000632,0.376908,-0.944062,0.000021
3,0,I11,57584.000000,30.854017,1.584372,0.225203,0.042807,2.933278,2.868369,2.854033,0.443083,-0.512143,-0.001835,0.009544,0.000638,0.392547,-0.937610,0.000021
4,0,I11,57585.000000,31.073293,1.626352,0.222233,0.041200,2.932822,2.854723,2.852182,0.452625,-0.511501,-0.001868,0.009539,0.000644,0.408074,-0.930896,0.000020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4,I41,57606.041667,81.604737,7.659941,0.796380,0.054431,1.062705,1.271248,0.885706,0.478627,-0.340272,-0.005148,0.014977,0.000038,0.701758,-0.732329,0.000053
296,4,I41,57607.041667,82.400711,7.714444,0.794552,0.052561,1.065133,1.271672,0.880449,0.493544,-0.340192,-0.005365,0.014858,0.000122,0.713800,-0.720380,0.000052
297,4,I41,57608.041667,83.194942,7.767081,0.792774,0.050691,1.067528,1.272003,0.874977,0.508341,-0.340028,-0.005579,0.014736,0.000205,0.725638,-0.708226,0.000051
298,4,I41,57609.041667,83.987478,7.817852,0.791046,0.048822,1.069890,1.272242,0.869293,0.523014,-0.339782,-0.005790,0.014611,0.000287,0.737269,-0.695873,0.000050
